In [5]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import csv
import os, sys

def get_fields(field_file):
    with open(field_file, 'r') as f:
        return list(csv.reader(f))[0]  # assume only one line


def fix_birthdays(x):
    try:
        return datetime.datetime.strptime(x, '%Y%m%d')
    except (ValueError, TypeError):
        return pd.NaT

In [7]:
df = pd.read_csv('../data/example_arrests.csv', dtype={'CRRKCY': str})
fields = pd.read_csv('../data/fields.csv', index_col='field')
marijuana_offense_codes = pd.read_csv(
    '../data/offense_codes_marijuana.csv')['code'].tolist()
cocaine_offense_codes = pd.read_csv(
    '../data/offense_codes_cocaine.csv')['code'].tolist()
heroin_offense_codes = pd.read_csv(
    '../data/offense_codes_heroin.csv')['code'].tolist()
county_codes = pd.read_csv(
    '../data/county_codes.csv', dtype={'code': str}).set_index('code')['county'].to_dict()

df = df[fields.index.tolist()]  # limit data frame to desired fields
df.rename(columns=fields['description'].to_dict(), inplace=True)
df['County'] = df['County_Number'].map(county_codes)

# Limit to different types of drug offenses
marijuana = df[df['Charged_Offense_Code'].isin(marijuana_offense_codes)]
cocaine = df[df['Charged_Offense_Code'].isin(cocaine_offense_codes)]
heroin = df[df['Charged_Offense_Code'].isin(heroin_offense_codes)]